In [2]:
#Отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# Подключаем графические модули:
# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# будем отображать графики прямо в jupyter'e
%pylab inline

#графики в svg выглядят более четкими
#%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from matplotlib import pyplot


#Подключаем модули для работы с ML
import numpy as np
import pandas as pd

#Для моделей
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error

from scipy import stats


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

#from sklearn.learning_curve import validation_curve
#from sklearn.learning_curve import learning_curve
from sklearn.metrics import accuracy_score ,roc_auc_score


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

from collections.abc import Iterable
from itertools import product

import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
#from sklearn.impute import SimpleImputer
#from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

#from sklearn.learning_curve import validation_curve
#from sklearn.learning_curve import learning_curve
from sklearn.metrics import accuracy_score ,roc_auc_score


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


from collections.abc import Iterable
from itertools import product
import time

import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 



Populating the interactive namespace from numpy and matplotlib


In [3]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return pd.DataFrame(X[self.columns])
    

    
class ModifiedSimpleImputer(SimpleImputer):
    
#     def fit_transform(self, X,y=None,  **fit_params):
#         self.fit(X,y,  **fit_params)
#         return self.transform(X)

    
    def transform(self, X):
        return pd.DataFrame(super().transform(X))
    

class ModifiedFeatureUnion(FeatureUnion):
    
#     def fit_transform(self, X,y=None,  **fit_params):
#         self.fit(X,y,  **fit_params)
#         return self.transform(X)
    def merge_dataframes_by_column(self, X):
        return pd.concat(X, axis="columns", copy=False)
    
    def transform(self, X):
        #X = self.merge_dataframes_by_column(X)
        return pd.DataFrame(super().transform(X))
    
    
class MyLEncoder():
    
    def transform(self, X, **fit_params):
        enc = LabelEncoder()
        enc_data = []
        for i in list(X.columns):
            X[i] = X[i].astype(str)
            encc = enc.fit(X[i])
            enc_data.append(encc.transform(X[i]))
        return np.asarray(enc_data).T
    
    def fit_transform(self, X,y=None,  **fit_params):
        self.fit(X,y,  **fit_params)
        return self.transform(X)
    def fit(self, X, y, **fit_params):
        return self 

In [4]:
class BaseAutoMlEstimator:
    """Base class for all estimators in scikit-learn.
    Notes
    -----
    All estimators should specify all the parameters that can be set
    at the class level in their ``__init__`` as explicit keyword
    arguments (no ``*args`` or ``**kwargs``).
    """
    
    def __init__(self, df, X_train, X_test, y_train, y_test, target_col, reports_path='/reports'):
        self.df = df
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.target_col = target_col        
        self.reports_path = reports_path

        
    @classmethod
    def get_params_combination(self, param_grid):
        iterator = product(*[v if isinstance(v, Iterable) else [v] for v in param_grid.values()])
        return [dict(zip(param_grid.keys(), values)) for values in iterator]
    
    
    def get_report_about_features(self,df, reports_path='/reports'):
        
        ### Формируем отчет по фичам DF
        dict_null = df.isnull().sum().to_dict()
        dict_type = df.dtypes.to_dict()

        dict_unique = dict()
        for column in df.columns:
            dict_unique[column] = len(df[column].unique())

        report_df = df.describe(include='all').T.rename_axis('features').reset_index()
        report_df['feat_count_null'] = report_df['features'].map(dict_null)
        report_df['feat_type'] = report_df['features'].map(dict_type)
        report_df['unique'] = report_df['features'].map(dict_unique)
        
        report_df.to_csv('..'+ reports_path + '/features_report.csv', index=False)   
        
        print("Отчет по статистикам по фичам сформирован")
        return report_df

    
    
    def optimize_types(self, df, inplace=False):
    
        np_types = [np.int8 ,np.int16 ,np.int32, np.int64,
                np.uint8 ,np.uint16, np.uint32, np.uint64]

        np_types = [np_type.__name__ for np_type in np_types]
        type_df = pd.DataFrame(data=np_types, columns=['class_type'])

        type_df['min_value'] = type_df['class_type'].apply(lambda row: np.iinfo(row).min)
        type_df['max_value'] = type_df['class_type'].apply(lambda row: np.iinfo(row).max)
        type_df['range'] = type_df['max_value'] - type_df['min_value']
        type_df.sort_values(by='range', inplace=True)


        for col in df.loc[:, df.dtypes <= np.integer]:
            col_min = df[col].min()
            col_max = df[col].max()
            temp = type_df[(type_df['min_value'] <= col_min) & (type_df['max_value'] >= col_max)]
            optimized_class = temp.loc[temp['range'].idxmin(), 'class_type']
            print("Название колонки : {} Минимальное значение : {} Максимальное значение : {} Можно оптимизировать к  : {}"\
                  .format(col, col_min, col_max, optimized_class))

            if inplace == 'True':
                df[col] = df[col].astype(optimized_class)

        #df.info()

        #return df


In [5]:
class AutoMlClassification(BaseAutoMlEstimator):    
    
    def __init__(self, df, X_train, X_test, y_train, y_test, target_col, reports_path='/reports'):
        super().__init__(df, X_train, X_test, y_train, y_test, target_col, reports_path='/reports')
        
    
    def get_report_about_target(self, df, target_col ,reports_path='/reports'):
        
        ### Формируем отчет по таргету DF
        
        target_report = df.groupby(target_col).count().reset_index()

        dict_count_traget = df[target_col].value_counts().to_dict()
        dict_count_traget_norm = df[target_col].value_counts(normalize=True).to_dict()

        target_report['count'] = target_report[target_col].map(dict_count_traget)
        target_report['count_norm'] = target_report[target_col].map(dict_count_traget_norm)
        
        target_report.to_csv('..'+ reports_path + '/target_report.csv', index=False)    
        
        print("Отчет по статистикам по таргету сформирован")
        return target_report
    
    
    
    def fit_report(self, X_train, X_test, y_train, y_test, reports_path='/reports'):
        
        digits_features = X_train.select_dtypes(include=['number']).columns.values.tolist()
        cat_features = X_train.select_dtypes(include=['object', ]).columns.values.tolist()
        
        classifiers = [LogisticRegression,
               #KNeighborsClassifier,
               #GradientBoostingClassifier(), 
               RandomForestClassifier] 
        #               SVC()] # 

        classifiers_name = ['LogisticRegression',

                            #'KNeighborsClassifier',
                            #'GradientBoostingClassifier', 
                            'RandomForestClassifier'] 
        #                    'SVC']


        # Настройка параметров выбранных алгоритмов с помощью GridSearchCV 
        n_folds = 5
        scores = []
        fits = []
        logistic_params = {'penalty': ('l1', 'l2'),
                           'C': (.01,5)}

        knn_params = {'n_neighbors': list(range(3, 6, 2))}


        gbm_params = {'n_estimators': [100, 300, 500],
                      'learning_rate':(0.1, 0.5, 1),
                      'max_depth': list(range(3, 6)), 
                      'min_samples_leaf': list(range(10, 31, 10))}



        forest_params = {'n_estimators': [10, 30],
                         'criterion': ('gini', 'entropy')}

        #svm_param = {'kernel' : ('linear', 'rbf'), 'C': (.5, 1, 2)} - очень долго считал
        #params = [logistic_params, knn_params, gbm_params, forest_params]

        params = [logistic_params, forest_params]        



        ############# Заполнение NaN #############
        imputer_numeric_list = [ModifiedSimpleImputer]
        imputer_numeric_name = ['ModifiedSimpleImputer'] 

        simpleimputer_params_numeric = {'fill_value': [0, -1],
                                        'strategy': ['constant']}
        params_numeric_list = [simpleimputer_params_numeric]        





        imputer_cat_list = [ModifiedSimpleImputer]
        imputer_cat_name = ['ModifiedSimpleImputer'] 

        simpleimputer_params_cat = {'strategy': ['constant'],
                                   'fill_value': ['missing']}
        params_cat_list = [simpleimputer_params_cat]        

        ############# Заполнение NaN #############



        ############# Scaler ################

        scaler_list = [MinMaxScaler]
        scaler_name_list = ['MinMaxScaler'] 

        scaler_params = {'feature_range': [(0,1) , (2,3)]}
        params_scaler_list = [scaler_params]        


        #####################################


        np.random.seed(0)




        np.random.seed(0)

        df1 = pd.DataFrame()

        skf = StratifiedKFold(n_splits=3, random_state=0)


        # for i , each_imputer_cat in enumerate(imputer_cat_list):
        #     imputer_cat = each_imputer_cat
        #     imputer_cat_name = imputer_cat_name[i]
        #     imputer_cat_params = params_cat_list[i]
        #     print("imputer_cat_name", imputer_cat_name)    
        #     for tmp_imputer_cat_params in get_params_combination(imputer_cat_params):
        #         print("Параметры: ", tmp_imputer_cat_params)


        for i , each_imputer_numeric in enumerate(imputer_numeric_list):
            imputer_numeric = each_imputer_numeric
            imputer_numeric_name = imputer_numeric_name[i]
            imputer_numeric_params = params_numeric_list[i]
            print("imputer_numeric_name fill_na", imputer_numeric_name)    
            for current_imputer_numeric_params in self.get_params_combination(imputer_numeric_params):
                print('\n', "Параметры: ", current_imputer_numeric_params)


                for i , each_scaler in enumerate(scaler_list):
                    scaler = each_scaler
                    scaler_name = scaler_name_list[i]
                    scaler_params = params_scaler_list[i]
                    print("scaler_name", scaler_name,)    
                    for current_scaler_param in self.get_params_combination(scaler_params):
                        print('\n', "Параметры scaler: ", current_scaler_param, '\n')

                        for i , each_classifier in enumerate(classifiers):
                            clf = each_classifier
                            clf_params = params[i]
                            clf_classifiers_name = classifiers_name[i]
                            print("classifiers_name", clf_classifiers_name,)

                            for tmp_params in self.get_params_combination(clf_params):
                                print("Параметры classifiers: ", tmp_params)
                                skf_index = skf.split(X_train, y_train)
                                for fold, (train_idx, test_idx) in enumerate(skf_index):
                                    print("Размер тренировочного / тестового датасета: ", len(train_idx), len(test_idx))

                                    # Формируем тренеровочный и валидационный датасет
                                    X_train_tmp, X_test_tmp = X_train.iloc[train_idx], X_train.iloc[test_idx]
                                    y_train_tmp, y_test_tmp = y_train.iloc[train_idx], y_train.iloc[test_idx]

                                    # Получаем модель
                                    #tmp_clf = clf(**tmp_params)

                                    tmp_clf = Pipeline([
                                            # Use FeatureUnion to combine the features
                                            ('union', ModifiedFeatureUnion(
                                                transformer_list=[
                                                     # categorical features
                                                    ('categorical', Pipeline([
                                                         ('selector', FeatureSelector(columns = cat_features)),
                                                         ('imputer', ModifiedSimpleImputer(strategy='constant', fill_value='missing')),
                                                         ('label_encoding', MyLEncoder())
                                                    ])),
                                                    # numeric features
                                                    ('numeric', Pipeline([
                                                         ('selector', FeatureSelector(columns = digits_features)),
                                                         ('imputer', imputer_numeric(**current_imputer_numeric_params)),
                                                         ('scaler', scaler(**current_scaler_param))
                                                    ])),
                                                ])),
                                            # Use model fit
                                            ('model_fitting', clf(**tmp_params)),
                                        ])



                                    # Замеряем время fit
                                    start_time = time.time()
                                    pred = tmp_clf.fit(X_train_tmp, y_train_tmp)
                                    fit_time = time.time() - start_time


                                    # Замеряем время predict
                                    start_time = time.time()
                                    pred = tmp_clf.predict(X_test_tmp)
                                    predict_time = time.time() - start_time

                                    clf_tmp_params_string = ", ".join(("{}={}".format(*i) for i in tmp_params.items()))
                                    scale_tmp_params_string = ", ".join(("{}={}".format(*i) for i in current_scaler_param.items()))
                                    imputer_numeric_params_string = ", ".join(("{}={}".format(*i) for i in current_imputer_numeric_params.items()))


                                    data = {'classifier_name' : clf_classifiers_name,
                                            'classifier_params' : clf_tmp_params_string,
                                            'scaler_name': scaler_name,
                                            'scaler_params': scale_tmp_params_string,

                                            'imputer_name': imputer_numeric_name,
                                            'imputer_params': imputer_numeric_params_string,

                                            'fold' : fold,
                                            'fit_time' : fit_time, 
                                            'predict_time' : predict_time,
                                            'roc_auc':roc_auc_score(y_test_tmp, pred)

                                    }

                                    # Расширяем другими параметрами
                                    data.update(tmp_params) # параметры классификатора
                                    data.update(current_scaler_param) # параметры scale
                                    data.update(current_imputer_numeric_params)

                                    # Формируем финальный датафрейм
                                    df1 = df1.append(data, ignore_index=True)

                    
        df1.to_csv('..'+ reports_path + '/model_report.csv', index=False)  
        print("Отчет по модели сформирован")

    

# Обработка данных

In [6]:
# Считываем данные и разбиваем их на Train и Test

df = pd.read_csv('../ml_data/train.csv')
target_col = 'Survived'

y = df[target_col]
X = df.drop([target_col], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [8]:
# Создаем экземпляр класса

#reports_path='/reports'
au1 = AutoMlClassification(df, X_train, X_test, y_train, y_test, target_col)

In [9]:
# Формируем отчет по таргету

au1.get_report_about_target(df, target_col)

Отчет по статистикам по таргету сформирован


,Survived,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,count,count_norm
0,0,549,549,549,549,424,549,549,549,549,68,549,549,0.616162
1,1,342,342,342,342,290,342,342,342,342,136,340,342,0.383838


# Черновик

In [ ]:
def count(func):
    def wrapper(self):
        func(self)
    return wrapper

In [20]:
from functools import wraps
class Repeater:
    def __init__(self, n):
        self.n = n

    def __call__(self, f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            for _ in range(self.n):
                f(*args, **kwargs)
        return wrapper
       
    
@Repeater(3)
def foo():
    print('foo')


foo()

foo
foo
foo


In [13]:
@Repeater.__test__()
def foo():
    print('foo')

TypeError: __test__() missing 2 required positional arguments: 'self' and 'f'

In [14]:
def count(func):
    def wrapper(self):
        TestClass.call_count += 1
        func(self)

    return wrapper


class TestClass(object):
    call_count = 0

    @count
    def hello(self):
        return 'hello'


x = TestClass()
for i in range(10):
    x.hello()

print(TestClass.call_count)

10


In [ ]:
# 1) Spark + Teradata

data = spark.read.jdbc("sbx_datamart.card_auths")

data.filter(col("city") == "IRKUTSK").select("*").write...
# Failed to perform, kicked by DBA
# in option query
# https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html


# 2) Spark SQL part
trans = spark.table("fraud.events")
cards = spark.table("fraud.bank_cards")

"""
trans:
    event_id
    user_id
    trans_date
    card_number
    event_amount
    merchant_name
    mcc
    

cards:
    card_number
    user_id
    office_id
    org_office
    segment
    
"""

datamart = trans.join(cards, ["card_number"])\
    .groupBy("user_id", "trans_date")\
    .agg(avg("event_amount").as("per_user_avg"))
    .filter(col("per_user_avg") > 123000)

# Failed to perform, OutOfMemoryError


In [ ]:
)  Множим таблицу  card с дополнительным полем id куда ставим цифры от 1 до 10.  те будет 10 таблицы card и у них в каждой будет одно значение id.  

2) Далее добавлемя к trans колонку id и туда рандомно ставим номер от 1 до 10. 

Далее делаем джойн, благодаря такому моменту получается, что джойн идет равномерно на экзекютеры.

Делается пункт 1) с помощью функции exploed

Также скошенность здесь по card_number  skew

In [ ]:
https://docs.databricks.com/delta/join-performance/skew-join.html